In [1]:
import pandas as pd
import matplotlib.pyplot as plt


In [32]:
def load_and_plot_csv(csv_path, x_column, y_column, figsize=(10, 6), nrows=1, ncols=1):
    try:
        # Load CSV data into a DataFrame
        df = pd.read_csv(csv_path)

        # Display the first few rows of the DataFrame (optional)
        print("Data Preview:")
        print(df.head())

        # Create subplots
        fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=figsize)

        # Plot the specific columns
        if nrows == 1 and ncols == 1:
            # Single plot
            df.plot(x=x_column, y=y_column, kind='line', title=f'Plot of {y_column} against {x_column}', ax=ax)
            ax.set_xlabel(f'{x_column} Axis Label')
            ax.set_ylabel(f'{y_column} Axis Label')
        else:
            # Multiple subplots
            for i in range(nrows):
                for j in range(ncols):
                    subset_df = df.sample(frac=1)  # Sample to get a subset of data
                    subset_df.plot(x=x_column, y=y_column, kind='line', title=f'Plot of {y_column} against {x_column}', ax=ax[i, j])
                    ax[i, j].set_xlabel(f'{x_column} Axis Label')
                    ax[i, j].set_ylabel(f'{y_column} Axis Label')

        plt.tight_layout()
        plt.show()

    except Exception as e:
        print(f"An error occurred: {e}")

In [35]:

csv_path = '../data/data(3).csv'
x_column_to_plot = 'Time(UTC+0)'  # Check the case and spelling in your CSV file
y_column_to_plot = 'PM2.5(μm/m^3)'
load_and_plot_csv(csv_path, x_column_to_plot, y_column_to_plot, figsize=(15, 10), nrows=3, ncols=100)

Data Preview:
           Time(UTC+0)          Time(UTC+8)  Temp(°C)  RH(%)  PM2.5(μm/m^3)  \
0  2024/03/12 05:57:51  2024/03/12 13:57:51     34.14  46.69             28   
1  2024/03/12 05:57:40  2024/03/12 13:57:40     34.07  46.78             27   
2  2024/03/12 05:57:30  2024/03/12 13:57:30     34.12  46.54             27   
3  2024/03/12 05:57:20  2024/03/12 13:57:20     34.11  46.79             27   
4  2024/03/12 05:57:10  2024/03/12 13:57:10     34.12  46.87             29   

   CO2(ppm)  
0       474  
1       476  
2       477  
3       477  
4       475  
